In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
pd.set_option('display.max_columns',None)

C:\Users\OMEN\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
traffic = pd.read_excel("dataset/traffic_data_2013-23.xlsx")
weather = pd.read_excel("dataset/weather_data_2013-23.xlsx")

print(traffic.shape, weather.shape)

(290705, 5) (86184, 8)


### Traffic EDA

In [3]:
traffic.head()

,Site Name,Report Date,Time Period Ending,Avg mph,Total Volume
0,7004/1,2013-06-01T00:00:00,00:59:00,NaN,69.0
1,7004/1,2013-06-01T00:00:00,01:59:00,NaN,33.0
2,7004/1,2013-06-01T00:00:00,02:59:00,NaN,23.0
3,7004/1,2013-06-01T00:00:00,03:59:00,NaN,24.0
4,7004/1,2013-06-01T00:00:00,04:59:00,NaN,26.0


In [4]:
traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290705 entries, 0 to 290704
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Site Name           290705 non-null  object 
 1   Report Date         290705 non-null  object 
 2   Time Period Ending  290705 non-null  object 
 3   Avg mph             255650 non-null  float64
 4   Total Volume        271224 non-null  float64
dtypes: float64(2), object(3)
memory usage: 11.1+ MB


In [5]:
traffic.isnull().mean()*100

Site Name              0.000000
Report Date            0.000000
Time Period Ending     0.000000
Avg mph               12.058616
Total Volume           6.701295
dtype: float64

In [6]:
traffic['Date'] = pd.to_datetime(traffic['Report Date'])
traffic['Hour'] = pd.to_timedelta(traffic['Time Period Ending'].str.slice(0, 2) + ':00:00')
traffic['Datetime'] = traffic['Date'] + traffic['Hour']

In [7]:
traffic.drop(['Site Name','Report Date','Time Period Ending','Avg mph','Date','Hour'], axis=1, inplace=True)

In [8]:
agg_traffic = traffic.groupby('Datetime')['Total Volume'].sum().reset_index()

In [9]:
agg_traffic.head()

,Datetime,Total Volume
0,2013-06-01 00:00:00,69.0
1,2013-06-01 01:00:00,33.0
2,2013-06-01 02:00:00,23.0
3,2013-06-01 03:00:00,24.0
4,2013-06-01 04:00:00,26.0


In [10]:
agg_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84349 entries, 0 to 84348
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Datetime      84349 non-null  datetime64[ns]
 1   Total Volume  84349 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.3 MB


In [11]:
agg_traffic = agg_traffic.set_index('Datetime')
date_range = pd.date_range(start=agg_traffic.index.min(), end=agg_traffic.index.max(), freq='H')
agg_traffic = agg_traffic.reindex(date_range, fill_value=0)
agg_traffic.index.names = ['Datetime']

In [12]:
agg_traffic

,Total Volume
Datetime,
2013-06-01 00:00:00,69.0
2013-06-01 01:00:00,33.0
2013-06-01 02:00:00,23.0
2013-06-01 03:00:00,24.0
2013-06-01 04:00:00,26.0
...,...
2023-03-31 20:00:00,233.0
2023-03-31 21:00:00,214.0
2023-03-31 22:00:00,162.0


In [13]:
sample = agg_traffic[:1000]

In [22]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import DatetimeTickFormatter

output_notebook()

p = figure(title='Time Series Data', x_axis_label='Datetime', y_axis_label='Total Volume', 
           x_axis_type='datetime', plot_width=1200, plot_height=600)

p.line(agg_traffic.index, agg_traffic['Total Volume'], line_width=2)

p.xaxis.formatter = DatetimeTickFormatter()
p.xaxis.major_label_orientation = 45

show(p)

Loading BokehJS ...